In [57]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd

In [58]:
df_states_original = pd.read_csv("./us-states.csv", dtype={"date": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
df_states_original.sort_values("fips", ascending=True)
# #Replaces any nan to 0
# if df_states_original.isnull().values.any():
#     df_states_original.replace(np.nan, None)

,date,state,fips,cases,deaths
2929,2020-04-25,Alabama,01,6213,213
7714,2020-07-21,Alabama,01,70358,1303
3644,2020-05-08,Alabama,01,9385,383
586,2020-03-13,Alabama,01,6,0
6174,2020-06-23,Alabama,01,31097,864
...,...,...,...,...,...
1275,2020-03-25,Virgin Islands,78,17,0
5618,2020-06-12,Virgin Islands,78,72,6
1221,2020-03-24,Virgin Islands,78,17,0
3088,2020-04-27,Virgin Islands,78,59,4


In [59]:
pivot_states = pd.pivot_table(df_states_original , index=['state','fips'],
                       columns=['date'])
if pivot_states.isnull().values.any():
    pivot_states = pivot_states.replace(np.nan, 0).astype(int)
pivot_states.head(5)

cases                                              \
date            2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state      fips                                                          
Alabama    01            0          0          0          0          0   
Alaska     02            0          0          0          0          0   
Arizona    04            0          0          0          0          0   
Arkansas   05            0          0          0          0          0   
California 06            0          0          0          0          1   

                                                                        ...  \
date            2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30  ...   
state      fips                                                         ...   
Alabama    01            0          0          0          0          0  ...   
Alaska     02            0          0          0          0          0  ...   
Arizona    04            1          1          1          1          1  ...   
Arkansas   05            0          0          0          0          0  ...   
California 06            2          2          2          2          2  ...   

                    deaths                                              \
date            2020-07-18 2020-07-19 2020-07-20 2020-07-21 2020-07-22   
state      fips                                                          
Alabama    01         1286       1287       1291       1303       1364   
Alaska     02           16         16         16         16         17   
Arizona    04         2733       2768       2795       2918       2976   
Arkansas   05          357        357        363        374        380   
California 06         7697       7710       7764       7883       8038   

                                                                        
date            2020-07-23 2020-07-24 2020-07-25 2020-07-26 2020-07-27  
state      fips                                                         
Alabama    01         1397       1438       1456       1473       1491  
Alaska     02           17         17         18         18         19  
Arizona    04         3067       3156       3288       3313       3320  
Arkansas   05          386        394        399        401        408  
California 06         8190       8325       8428       8451       8544  

[5 rows x 378 columns]

Get Date Info

In [60]:
date= pivot_states['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       ...
       '2020-07-18', '2020-07-19', '2020-07-20', '2020-07-21', '2020-07-22',
       '2020-07-23', '2020-07-24', '2020-07-25', '2020-07-26', '2020-07-27'],
      dtype='object', name='date', length=189)

In [61]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [62]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

189 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30',
               ...
               '2020-07-18', '2020-07-19', '2020-07-20', '2020-07-21',
               '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-25',
               '2020-07-26', '2020-07-27'],
              dtype='datetime64[ns]', length=189, freq='D')
189 ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-

Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [63]:
# import json
# pivot_states['cases_ts'] = "'" + json.dumps({"values": pivot_states['cases'].values.tolist()[0]}) + "'"
# pivot_states['deaths_ts'] = "'" + json.dumps({"values": pivot_states['deaths'].values.tolist()[0]}) + "'"

pivot_states['cases_ts'] = pivot_states['cases'].values.tolist()
pivot_states['cases_ts'] = pivot_states['cases_ts'].apply(lambda x: ','.join(map(str, x)))
pivot_states['deaths_ts'] = pivot_states['deaths'].values.tolist()
pivot_states['deaths_ts'] = pivot_states['deaths_ts'].apply(lambda x: ','.join(map(str, x)))

In [64]:
yesterday = date.values[-2]
pivot_states['today_case'] = pivot_states['cases'][dt_str_end]
pivot_states['today_new_case'] = pivot_states['cases'][dt_str_end] - pivot_states['cases'][yesterday]
pivot_states['today_death'] = pivot_states['deaths'][dt_str_end]
pivot_states['today_new_death'] = pivot_states['deaths'][dt_str_end] - pivot_states['deaths'][yesterday]

In [65]:
pivot_states['dt_first_case'] = (pivot_states['cases'] > 0).idxmax(axis=1)
pivot_states['dt_first_death'] = (pivot_states['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths counties
pivot_states.loc[pivot_states['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_states.head(5)

cases                                              \
date            2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state      fips                                                          
Alabama    01            0          0          0          0          0   
Alaska     02            0          0          0          0          0   
Arizona    04            0          0          0          0          0   
Arkansas   05            0          0          0          0          0   
California 06            0          0          0          0          1   

                                                                        ...  \
date            2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30  ...   
state      fips                                                         ...   
Alabama    01            0          0          0          0          0  ...   
Alaska     02            0          0          0          0          0  ...   
Arizona    04            1          1          1          1          1  ...   
Arkansas   05            0          0          0          0          0  ...   
California 06            2          2          2          2          2  ...   

                    deaths             \
date            2020-07-26 2020-07-27   
state      fips                         
Alabama    01         1473       1491   
Alaska     02           18         19   
Arizona    04         3313       3320   
Arkansas   05          401        408   
California 06         8451       8544   

                                                          cases_ts  \
date                                                                 
state      fips                                                      
Alabama    01    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Alaska     02    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Arizona    04    0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...   
Arkansas   05    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
California 06    0,0,0,0,1,2,2,2,2,2,3,3,6,6,6,6,6,6,6,6,7,7,7,...   

                                                         deaths_ts today_case  \
date                                                                            
state      fips                                                                 
Alabama    01    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...      81115   
Alaska     02    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...       3204   
Arizona    04    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...     163918   
Arkansas   05    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...      39447   
California 06    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...     467103   

                today_new_case today_death today_new_death dt_first_case  \
date                                                                       
state      fips                                                            
Alabama    01             1821        1491              18    2020-03-13   
Alaska     02              104          19               1    2020-03-12   
Arizona    04             1877        3320               7    2020-01-26   
Arkansas   05              824         408               7    2020-03-11   
California 06             7765        8544              93    2020-01-25   

                dt_first_death  
date                            
state      fips                 
Alabama    01       2020-03-25  
Alaska     02       2020-03-27  
Arizona    04       2020-03-20  
Arkansas   05       2020-03-24  
California 06       2020-03-04  

[5 rows x 386 columns]

In [66]:
pivot_states['today_case'].sum()

4303813

Read County Boundary GeoJSON file

In [67]:
old_states_geojson_df = gpd.read_file(r"./states_update.geojson")
old_states_geojson_df.head(5)

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,...,d2020-03-19,d2020-03-20,d2020-03-21,d2020-03-22,d2020-03-23,d2020-03-24,d2020-03-25,d2020-03-26,population,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,...,0,0,0,0,0,0,0,0,1829054,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,...,1,1,2,2,4,3,0,6,20598139,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,...,3,1,1,1,5,4,3,7,12821497,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,...,0,0,1,0,0,0,0,1,5527358,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,...,0,1,1,0,0,1,0,0,6003435,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


In [68]:
old_states_geojson_df[['NAME','population','GEOID','geometry']]

,NAME,population,GEOID,geometry
0,West Virginia,1829054,54,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."
5,Rhode Island,1056611,44,"MULTIPOLYGON (((-71.67881 41.15891, -71.63497 ..."
6,Idaho,1687809,16,"POLYGON ((-116.91607 45.99536, -116.98259 46.0..."
7,New Hampshire,1343622,33,"POLYGON ((-71.50108 45.01335, -71.49804 45.070..."
8,North Carolina,10155624,37,"POLYGON ((-81.67749 36.58814, -80.12217 36.542..."
9,Vermont,624977,50,"POLYGON ((-73.34312 45.01084, -71.46505 45.013..."


Old Data Structure

In [69]:
report_df = pivot_states[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,01,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-13,2020-03-25,81115,1821,1491,18
1,Alaska,02,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-12,2020-03-27,3204,104,19,1
2,Arizona,04,"0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-01-26,2020-03-20,163918,1877,3320,7
3,Arkansas,05,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-11,2020-03-24,39447,824,408,7
4,California,06,"0,0,0,0,1,2,2,2,2,2,3,3,6,6,6,6,6,6,6,6,7,7,7,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-01-25,2020-03-04,467103,7765,8544,93


In [70]:
#Stay consistency with geojson states
report_df = report_df.replace('Northern Mariana Islands','Commonwealth of the Northern Mariana Islands')

In [71]:
final_df = pd.merge(old_states_geojson_df[["NAME", "population", "geometry"]], report_df, how='left', left_on=['NAME'], right_on = ['state'])
final_df.columns

Index(['NAME', 'population', 'geometry', 'state', 'fips', 'cases_ts',
       'deaths_ts', 'dt_first_case', 'dt_first_death', 'today_case',
       'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

In [72]:
final_df=final_df[["NAME", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 'today_case','today_new_case', 'today_death','today_new_death', "geometry"]]
final_df['start'] = dt_str_start
final_df['end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",6054.0,94.0,106.0,3.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-07-27,day
1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",432739.0,8892.0,5930.0,77.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-07-27,day
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",174442.0,1363.0,7619.0,24.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-07-27,day
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",51843.0,650.0,1616.0,2.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-07-27,day
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",85436.0,1130.0,3447.0,7.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-07-27,day


In [73]:
final_df[final_df['NAME'] == 'American Samoa']

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
41,American Samoa,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-171.14118 -11.04661, -171.111...",2020-01-21,2020-07-27,day


In [74]:
#New York Times remove Samoa data since 4/27
final_df = final_df.drop(final_df[final_df['NAME'] == 'American Samoa'].index)
final_df = final_df.reset_index()
final_df

,index,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",6054.0,94.0,106.0,3.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-07-27,day
1,1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",432739.0,8892.0,5930.0,77.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-07-27,day
2,2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",174442.0,1363.0,7619.0,24.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-07-27,day
3,3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",51843.0,650.0,1616.0,2.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-07-27,day
4,4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",85436.0,1130.0,3447.0,7.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-07-27,day
5,5,Rhode Island,1056611,44,2020-03-01,2020-03-28,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",18515.0,291.0,1004.0,2.0,"MULTIPOLYGON (((-71.67881 41.15891, -71.63497 ...",2020-01-21,2020-07-27,day
6,6,Idaho,1687809,16,2020-03-13,2020-03-26,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",18836.0,574.0,159.0,9.0,"POLYGON ((-116.91607 45.99536, -116.98259 46.0...",2020-01-21,2020-07-27,day
7,7,New Hampshire,1343622,33,2020-03-02,2020-03-23,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",6441.0,5.0,409.0,0.0,"POLYGON ((-71.50108 45.01335, -71.49804 45.070...",2020-01-21,2020-07-27,day
8,8,North Carolina,10155624,37,2020-03-03,2020-03-25,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",114689.0,1808.0,1815.0,8.0,"POLYGON ((-81.67749 36.58814, -80.12217 36.542...",2020-01-21,2020-07-27,day
9,9,Vermont,624977,50,2020-03-07,2020-03-19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1402.0,2.0,56.0,0.0,"POLYGON ((-73.34312 45.01084, -71.46505 45.013...",2020-01-21,2020-07-27,day


In [75]:
final_df['change_ts'] = ""
for x in range(0, len(final_df)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = final_df.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
#     print(','.join(changeRateStrList))
    final_df.loc[x,'change_ts'] = ','.join(changeRateStrList)

final_df

,index,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit,change_ts
0,0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",6054.0,94.0,106.0,3.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",432739.0,8892.0,5930.0,77.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",174442.0,1363.0,7619.0,24.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,-1.0,-1...."
3,3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",51843.0,650.0,1616.0,2.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",85436.0,1130.0,3447.0,7.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
5,5,Rhode Island,1056611,44,2020-03-01,2020-03-28,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",18515.0,291.0,1004.0,2.0,"MULTIPOLYGON (((-71.67881 41.15891, -71.63497 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
6,6,Idaho,1687809,16,2020-03-13,2020-03-26,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",18836.0,574.0,159.0,9.0,"POLYGON ((-116.91607 45.99536, -116.98259 46.0...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
7,7,New Hampshire,1343622,33,2020-03-02,2020-03-23,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",6441.0,5.0,409.0,0.0,"POLYGON ((-71.50108 45.01335, -71.49804 45.070...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
8,8,North Carolina,10155624,37,2020-03-03,2020-03-25,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",114689.0,1808.0,1815.0,8.0,"POLYGON ((-81.67749 36.58814, -80.12217 36.542...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
9,9,Vermont,624977,50,2020-03-07,2020-03-19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1402.0,2.0,56.0,0.0,"POLYGON ((-73.34312 45.01084, -71.46505 45.013...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


In [76]:
final_df.to_file(r"./nyt_states_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


New Data Structure (Full version)

In [77]:
# pivot_states.columns = ['_'.join(col).strip() for col in pivot_states.columns.values]
# report_df_2 = pivot_states.reset_index()

In [78]:
# report_df_2.head(5)

In [79]:
# pivot_states.head(5)

In [80]:
# final_df_2 = pd.merge(old_states_geojson_df[["NAME", "population", "geometry"]], report_df_2, how='right', left_on=['NAME'], right_on = ['state'])
# final_df_2 = final_df_2.drop(['state'], axis = 1)
# final_df_2['dt_start'] = dt_start
# final_df_2['dt_end'] = dt_end
# final_df_2.head(5)

In [81]:
# final_df_2.to_file(r"./nyt_states_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

